# Fast Pandas

Pandas is a high-performance library for doing data analysis in Python... if you use it correctly. Today we'll go through some common performance traps people fall into, and we'll see how to stay on the fast path.

## Measure twice, cut once

We know the dangers of [premature optimization](https://en.wikipedia.org/wiki/Program_optimization#When_to_optimize), so before you spend time speeding up some code, it's worth verifying that the code in question is *actually* slow, and identify exactly where it's slow. For this, I prefer tools like

* [snakeviz](https://jiffyclub.github.io/snakeviz/) for function-level profiling
* [line-profiler](https://github.com/pyutils/line_profiler) meauring specific functions line-by-line.

See https://tomaugspurger.github.io/maintaing-performance.html for more on how to use these tools to identify slow sections of code. From here no out, we'll assume you've already verified that some code needs optimizing.

## Storage formats and I/O

Your first interaction with pandas is often one of the `pd.read_<format>` functions. While pandas supports reading from many different formats, some are higher-performance than others. In particular, we'll compare two specific formats

* CSV
* Parquet

If your workload is IO-bound and if you're lucky enough to choose your storage format, switching your storage format can have a big speedup. Let's generate some data for comparison:

In [2]:
# %load utils.py
# A helper to generate some dummy data
import pandas as pd
import numpy as np

names = ["Alice", "Bob", "Charlie", "Dan", "Edith", "Frank", "George", "Hannah", "Ingrid", "Jerry", "Kevin", "Laura", "Michael", "Norbert", "Oliver", "Patricia", "Quinn", "Ray", "Sarah", "Tim", "Ursula", "Victor", "Wendy", "Xavier", "Yvonne", "Zelda",]


def make_float(n, rstate):
    return rstate.rand(n) * 2 - 1


def make_int(n, rstate, lam=1000):
    return rstate.poisson(lam, size=n)

def make_string(n, rstate):
    return rstate.choice(names, size=n)


def make_categorical(n, rstate):
    return pd.Categorical.from_codes(rstate.randint(0, len(names), size=n), names)


make = {
    float: make_float,
    int: make_int,
    str: make_string,
    object: make_string,
    "category": make_categorical,
}


def make_timeseries_part(
        start="2000-01-01",
        end="2000-01-31",
        dtypes={"name": "category", "id": int, "x": float, "y": float},
        freq="10s",
        random_state=None,
        kwargs=None
    ):
    kwargs = kwargs or {}
    index = pd.date_range(start=start, end=end, freq=freq, name="timestamp")
    state = np.random.RandomState(random_state)
    columns = {}
    for k, dt in dtypes.items():
        kws = {
            kk.rsplit("_", 1)[1]: v
            for kk, v in kwargs.items()
            if kk.rsplit("_", 1)[0] == k
        }
        columns[k] = make[dt](len(index), state, **kws)
    df = pd.DataFrame(columns, index=index, columns=sorted(columns))
    if df.index[-1] == end:
        df = df.iloc[:-1]
    return df


def make_timeseries(
    start="2000-01-01",
    end="2000-12-31",
    dtypes={"name": str, "id": int, "x": float, "y": float},
    freq="10s",
    partition_freq="1M",
    seed=None,
    **kwargs
):
    """Create timeseries dataframe with random data
    Parameters
    ----------
    start: datetime (or datetime-like string)
        Start of time series
    end: datetime (or datetime-like string)
        End of time series
    dtypes: dict
        Mapping of column names to types.
        Valid types include {float, int, str, 'category'}
    freq: string
        String like '2s' or '1H' or '12W' for the time series frequency
    partition_freq: string
        String like '1M' or '2Y' to divide the dataframe into partitions
    seed: int (optional)
        Randomstate seed
    kwargs:
        Keywords to pass down to individual column creation functions.
        Keywords should be prefixed by the column name and then an underscore.
    Examples
    --------
    >>> import dask.dataframe as dd
    >>> df = dd.demo.make_timeseries('2000', '2010',
    ...                              {'value': float, 'name': str, 'id': int},
    ...                              freq='2H', partition_freq='1D', seed=1)
    >>> df.head()  # doctest: +SKIP
                           id      name     value
    2000-01-01 00:00:00   969     Jerry -0.309014
    2000-01-01 02:00:00  1010       Ray -0.760675
    2000-01-01 04:00:00  1016  Patricia -0.063261
    2000-01-01 06:00:00   960   Charlie  0.788245
    2000-01-01 08:00:00  1031     Kevin  0.466002
    """
    divisions = list(pd.date_range(start=start, end=end, freq=partition_freq))
    state_data = random_state_data(len(divisions) - 1, seed)
    name = "make-timeseries-" + tokenize(
        start, end, dtypes, freq, partition_freq, state_data
    )
    dsk = {
        (name, i): (
            make_timeseries_part,
            divisions[i],
            divisions[i + 1],
            dtypes,
            freq,
            state_data[i],
            kwargs,
        )
        for i in range(len(divisions) - 1)
    }
    head = make_timeseries_part("2000", "2000", dtypes, "1H", state_data[0], kwargs)
    return DataFrame(dsk, name, head, divisions)

In [13]:
import pandas as pd
import numpy as np

# import utils

ts = make_timeseries_part()
ts

,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,990,Ray,-0.178968,0.567300
2000-01-01 00:00:10,1033,George,0.350858,0.577017
2000-01-01 00:00:20,960,Bob,-0.624726,0.245760
2000-01-01 00:00:30,1032,Frank,-0.007736,0.133900
2000-01-01 00:00:40,991,Ingrid,0.136349,-0.822735
...,...,...,...,...
2000-01-30 23:59:20,1046,Bob,0.247978,-0.692476
2000-01-30 23:59:30,1036,Quinn,-0.698200,0.329448
2000-01-30 23:59:40,1049,Jerry,-0.803369,0.884677


We'll write it to the two formats, first CSV and then parquet.

In [14]:
%time ts.to_csv("data.csv")

CPU times: user 1.33 s, sys: 54.2 ms, total: 1.38 s
Wall time: 1.4 s


In [15]:
%time ts.to_parquet("data.parquet")

CPU times: user 99.5 ms, sys: 54.1 ms, total: 154 ms
Wall time: 153 ms


Already we see that parquet can be faster at writing, at least for this dataset (it may be slower for small dataframes). But the difference is even more striking when you try to read that data back in.

CSV is a plaintext format. This can be nice if you want to visually inspect the file. However, it's often slower and (at least for CSV) lacks any way to store data types in the file itself. Let's read them back in with the default arguments.

In [16]:
%time csv = pd.read_csv("data.csv")

CPU times: user 239 ms, sys: 50.7 ms, total: 289 ms
Wall time: 289 ms


In [17]:
%time parquet = pd.read_parquet("data.parquet")

CPU times: user 24.1 ms, sys: 20.8 ms, total: 44.9 ms
Wall time: 39 ms


Parquet is about 5x faster. But more importantly, the data read from CSV doesn't exactly match what was written. The original dtypes are

In [18]:
ts.dtypes

id         int64
name    category
x        float64
y        float64
dtype: object

CSV read them back in as

In [19]:
csv.dtypes

timestamp     object
id             int64
name          object
x            float64
y            float64
dtype: object

And parquet as

In [20]:
parquet.dtypes

id         int64
name    category
x        float64
y        float64
dtype: object

When reading CSVs, pandas has to infer the dtypes. This is slow (especially for datetimes, so pandas doesn't infer datetimes by default) or impossible for more exotic types like Categorical. You'd need to store these types seperately and provide them explicitly.

In [21]:
%%time
dtype = {"name": pd.CategoricalDtype(names)}
csv = pd.read_csv(
    "data.csv",
    parse_dates=["timestamp"],
    dtype=dtype,
    index_col="timestamp"
)

CPU times: user 278 ms, sys: 49.6 ms, total: 328 ms
Wall time: 329 ms


So in addition to being faster than CSVs (at least beyond small datasets), parquet can better 
preserve the data types.

## Reading Parts of the file

When you're optitmizing some piece of code, the fastest way to do something is to not do it at all. Some readers (including both `read_csv` and `read_parquet`) have support for selecting subsets of the data for reading. Both `read_csv` and `read_parquet` let you select a subset of the columns to read in. By not having to read / parse other parts of the dataset, you speed up the reading of the parts you care about.

In [22]:
%time _ = pd.read_parquet("data.parquet", columns=["x", "y"])

CPU times: user 18.7 ms, sys: 15.1 ms, total: 33.8 ms
Wall time: 30.2 ms


`read_parquet` also supports selecting a subset of *rows* to read. For maximum performance, you'll want to partition the dataset on disk according to your access pattern. For example, if we want to select a subset of the names, we'd partition on `name`.

In [23]:
ts.to_parquet("data-split.parquet", partition_cols=["name"])

In [25]:
%%time
ts_frank = pd.read_parquet(
    "data-split.parquet",
    columns=["x", "y"],
    filters=[("name", "=", "Frank")]
)

CPU times: user 17.2 ms, sys: 12.7 ms, total: 29.9 ms
Wall time: 17.1 ms


We've gone from about 300ms to read the full dataset with CSV to about 14ms to read this subset with Parquet.

## Constructing DataFrames

A common pattern is to store a full dataset as a bunch of files on disk with the same structure. Suppose we have a directory of parquet files that are generated by some batch process that runs at the end of the month.

In [26]:
months = list(pd.date_range(start="2000-01-01", end="2010-12-31", freq="1M"))
for i in range(len(months) - 1):
    start, end = months[i], months[i + 1]
    df = make_timeseries_part(start, end, freq="5T")
    df.to_parquet(f"data/{start}.parquet")

In [27]:
ls data/ | head

2000-01-31 00:00:00.parquet
2000-02-29 00:00:00.parquet
2000-03-31 00:00:00.parquet
2000-04-30 00:00:00.parquet
2000-05-31 00:00:00.parquet
2000-06-30 00:00:00.parquet
2000-07-31 00:00:00.parquet
2000-08-31 00:00:00.parquet
2000-09-30 00:00:00.parquet
2000-10-31 00:00:00.parquet


And let's suppose we wanted to go from disk to a single pandas dataframe. We have two ways we could get there

1. Initialize one DataFrame and append to that.
2. Make many smaller DataFrames and concatenate them together at the end.

If you were using Python data structures (lists, dictionaries, sets) you'd probably use the first way. In pandas (and NumPy) the second route is faster. Let's compare.

In [28]:
import pathlib
files = list(pathlib.Path("data").glob("*.parquet"))

This is the first method: creating an empty DataFrame and appending to it. We'll see that it's relatively slow.

In [29]:
%%time
result = pd.DataFrame({
    "id": np.array([], dtype="int64"),
    "name": pd.Categorical([], categories=names),
    "x": np.array([], dtype="float64"),
    "y": np.array([], dtype="float64")
}, index=pd.DatetimeIndex([], name='timestamp'))


for file in files:
    df_part = pd.read_parquet(file)
    result = result.append(df_part)

CPU times: user 1.33 s, sys: 801 ms, total: 2.13 s
Wall time: 2.13 s


The second method.

In [30]:
%%time
parts = [pd.read_parquet(file) for file in files]
ts_full = pd.concat(parts)

CPU times: user 586 ms, sys: 188 ms, total: 774 ms
Wall time: 752 ms


So we have something like a 2X speedup, by simply reading first and then concatenating. Why is that?

pandas' `DataFrame.append` is modeled after Python's `list.append`, but memory-wise they're very differnt. Recall that the columns inside a pandas DataFrame are typically NumPy arrays, and these cannot be expanded inplace. Expanding a NumPy array really means copying the whole thing.

So repeatedly calling DataFrame.append means repeatedly copying a whole bunch of NumPy arrays.

## Data Types

Recent versions of pandas feature *nullable data types*. In addition to being more sound with the types of data, they can offer performance improvements.

For example, let's generate some boolean data with missing values.

In [31]:
s1 = pd.Series(
    np.array([True, False, np.nan], dtype=object)
).repeat(10000)
s1

0    True
0    True
0    True
0    True
0    True
     ... 
2     NaN
2     NaN
2     NaN
2     NaN
2     NaN
Length: 30000, dtype: object

Notice that `s1.dtype` is `object`. That's because NumPy doesn't have a boolean dtype that can store missing values. So pandas falls back to an object-dtype ndarray of *Python* objects, which don't benefit from NumPy's typically optizations. The memory usage will be higher, and operations will be slower.

We can use pandas' nullable boolean dtype by calling `pd.array()` or by specifying `dtype="boolean"`.

In [32]:
s2 = pd.Series(
    pd.array([True, False, pd.NA], dtype="boolean")
).repeat(10000)
s2

0    True
0    True
0    True
0    True
0    True
     ... 
2    <NA>
2    <NA>
2    <NA>
2    <NA>
2    <NA>
Length: 30000, dtype: boolean

When there are missing values, pandas' nullable boolean type takes less memory than the object dtype.

In [33]:
s1.memory_usage(deep=True) / s2.memory_usage(deep=True)

4.133333333333334

And operations (like reductions, comparisons, arithmetic, logical operations) take less time.

In [34]:
%timeit s1 | s1

100 loops, best of 5: 9.87 ms per loop


In [35]:
%timeit s2 | s2

The slowest run took 9.01 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 5: 84.2 µs per loop


Apparently something is being cached, but ignoring that we're still much faster using pandas' nullable type.

Likewise for reductions like `sum` or `mean`.

In [36]:
%timeit s1.sum()

1000 loops, best of 5: 1.8 ms per loop


In [37]:
%timeit s2.sum()

The slowest run took 6.31 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 5: 45 µs per loop


Spend some time ensuring that your dtypes look correct. You'll want to avoid `object` dtype whenever possible. Pandas is gradually adding new extension dtypes for more types of data, so object dtype should become rarer.

It's worth mentioning pandas' `Categorical` dtype. This is a "dictionary encoded" type, where we store the unique set of allowed values once (`.categories`) and the specific value for a row as a compressed integer (`.codes`). This gives lower memory usage and (sometimes) faster operations.

For example, `name` is a Categorical storing strings. Let's compare operations on it with an `object` dtype version.

In [38]:
name = ts['name']
name_obj = name.astype(object)
display(name)

timestamp
2000-01-01 00:00:00        Ray
2000-01-01 00:00:10     George
2000-01-01 00:00:20        Bob
2000-01-01 00:00:30      Frank
2000-01-01 00:00:40     Ingrid
                        ...   
2000-01-30 23:59:20        Bob
2000-01-30 23:59:30      Quinn
2000-01-30 23:59:40      Jerry
2000-01-30 23:59:50     Oliver
2000-01-31 00:00:00    Michael
Freq: 10S, Name: name, Length: 259201, dtype: category
Categories (26, object): ['Alice', 'Bob', 'Charlie', 'Dan', ..., 'Wendy', 'Xavier', 'Yvonne', 'Zelda']

First of all, the object-dtype version uses more memory.

In [39]:
name_obj.memory_usage(deep=True) / name.memory_usage(deep=True)

7.806180561799196

Operations like `.value_counts()` are faster on the categorical version.

In [40]:
%timeit name_obj.value_counts()

100 loops, best of 5: 15.2 ms per loop


In [41]:
%timeit name.value_counts()

1000 loops, best of 5: 1.26 ms per loop


Be warned that `Categorical` isn't purely an optimization. It does change the semantics of some operations (especially around ordering and "unobserved" categories).

## Iteration, Apply, and Vectorization

One of the keys to achieving high-performance in Pandas (and Python, genenerally) is to avoid doing too much in Python. We want to push the computationally expensive pieces down to compiled languages like C.

Let's suppose we have some data on airports, and wanted to compute the pairwise distances between each.

In [42]:
airports = pd.read_csv(
    "https://vega.github.io/vega-datasets/data/airports.csv",
    index_col="iata",
    nrows=500,
)
airports

,name,city,state,country,latitude,longitude
iata,,,,,,
00M,Thigpen,Bay Springs,MS,USA,31.953765,-89.234505
00R,Livingston Municipal,Livingston,TX,USA,30.685861,-95.017928
00V,Meadow Lake,Colorado Springs,CO,USA,38.945749,-104.569893
01G,Perry-Warsaw,Perry,NY,USA,42.741347,-78.052081
01J,Hilliard Airpark,Hilliard,FL,USA,30.688012,-81.905944
...,...,...,...,...,...,...
57B,Islesboro,Islesboro,ME,USA,44.302856,-68.910587
57C,East Troy Municipal,East Troy,WI,USA,42.797111,-88.372500
59B,Newton,Jackman,ME,USA,45.631991,-70.247289


We'll do a bit of renaming and reindexing to generate the DataFrame of pairs.

In [43]:
columns = ["longitude", "latitude"]
idx = pd.MultiIndex.from_product([airports.index, airports.index],
                                 names=['orig', 'dest'])

pairs = pd.concat([
    airports[columns]
        .add_suffix('_orig')
        .reindex(idx, level='orig'),
    airports[columns]
        .add_suffix('_dest')
        .reindex(idx, level='dest')
    ], axis="columns"
)
pairs

longitude_orig  latitude_orig  longitude_dest  latitude_dest
orig dest                                                              
00M  00M       -89.234505      31.953765      -89.234505      31.953765
     00R       -89.234505      31.953765      -95.017928      30.685861
     00V       -89.234505      31.953765     -104.569893      38.945749
     01G       -89.234505      31.953765      -78.052081      42.741347
     01J       -89.234505      31.953765      -81.905944      30.688012
...                   ...            ...             ...            ...
5A6  57B       -89.729248      33.465401      -68.910587      44.302856
     57C       -89.729248      33.465401      -88.372500      42.797111
     59B       -89.729248      33.465401      -70.247289      45.631991
     5A4       -89.729248      33.465401      -88.726189      34.015805
     5A6       -89.729248      33.465401      -89.729248      33.465401

[250000 rows x 4 columns]

And now let's consider two implementations of the great circle distance computation.

The first will use pure Python, and computes the distance between two points.

In [45]:
import math


def gcd_py(lat1, lng1, lat2, lng2):
    '''
    Calculate great circle distance between two points.
    https://www.johndcook.com/blog/python_longitude_latitude/

    Parameters
    ----------
    lat1, lng1, lat2, lng2: float

    Returns
    -------
    distance:
      distance from ``(lat1, lng1)`` to ``(lat2, lng2)`` in kilometers.
    '''
    # python2 users will have to use ascii identifiers (or upgrade)
    degrees_to_radians = math.pi / 180.0
    ϕ1 = (90 - lat1) * degrees_to_radians
    ϕ2 = (90 - lat2) * degrees_to_radians

    θ1 = lng1 * degrees_to_radians
    θ2 = lng2 * degrees_to_radians

    cos = (math.sin(ϕ1) * math.sin(ϕ2) * math.cos(θ1 - θ2) +
           math.cos(ϕ1) * math.cos(ϕ2))
    # round to avoid precision issues on identical points causing ValueErrors
    cos = round(cos, 8)
    arc = math.acos(cos)
    return arc * 6373  # radius of earth, in kilometers

The second uses NumPy, and computes the distances between *arrays* of points.
Notice how similar the two implementations are.

In [46]:
def gcd_vec(lat1, lng1, lat2, lng2):
    '''
    Calculate great circle distance.
    https://www.johndcook.com/blog/python_longitude_latitude/

    Parameters
    ----------
    lat1, lng1, lat2, lng2: float or array of float

    Returns
    -------
    distance:
      distance from ``(lat1, lng1)`` to ``(lat2, lng2)`` in kilometers.
    '''
    ϕ1 = np.deg2rad(90 - lat1)
    ϕ2 = np.deg2rad(90 - lat2)

    θ1 = np.deg2rad(lng1)
    θ2 = np.deg2rad(lng2)

    cos = (np.sin(ϕ1) * np.sin(ϕ2) * np.cos(θ1 - θ2) +
           np.cos(ϕ1) * np.cos(ϕ2))
    # round to avoid precision issues on identical points causing warnings
    cos = np.round(cos, 8)
    arc = np.arccos(cos)
    return arc * 6373 # radius of earth, in kilometers

And now let's use these functions in a few different ways.

1. Pass `gcd_py` to `DataFrame.apply`
2. Manually iterate over the DataFrame, calling `gcd_py` on each row
3. Call `gcd_vec`.

In [47]:
%%time
# gcd_py with DataFrame.apply
r = pairs.apply(
    lambda x: gcd_py(x['latitude_orig'],
                     x['longitude_orig'],
                     x['latitude_dest'],
                     x['longitude_dest']),
                axis="columns"
);

CPU times: user 4.92 s, sys: 69.2 ms, total: 4.99 s
Wall time: 5.01 s


In [48]:
%%time
# gcd_py with manual iteration
_ = pd.Series([gcd_py(*x) for x in pairs.itertuples(index=False)],
              index=pairs.index)

CPU times: user 616 ms, sys: 16.9 ms, total: 633 ms
Wall time: 633 ms


In [49]:
%%time
# gcd_vec
r = gcd_vec(pairs['latitude_orig'], pairs['longitude_orig'],
            pairs['latitude_dest'], pairs['longitude_dest'])

CPU times: user 162 ms, sys: 11 ms, total: 173 ms
Wall time: 174 ms


In [50]:
r.head()

orig  dest
00M   00M        0.000000
      00R      567.271820
      00V     1589.259385
      01G     1551.898663
      01J      710.296324
dtype: float64

Performance-wise, it's clear that the vectorized version is best. And, in my opinion, the code is clearer.

DataFrame.apply is the clear loser hear. It can be useful for quickly writing some little transformation. But `DataFrame.apply(..., axis=1)` generally should be avoided, especially for performance-sensitive code. It does much more work than the other forms we showed.

Not every problem can be solved with vecorization though. Some problems are difficult or impossible to express using just Numpy. For those, we fortunately have Numba. 

## Using Numba

Recent versions of pandas optionally make extensive use of Numba to speed up certain operations. This is helpful when you have some custom user-defined function that you're passing to one of pandas' `.apply`, `.agg`, or `.transform` methods (in a rolling or groupby context).

Consider something like a `df.rolling(n).apply(func)`. At a high level, that operation requires

1. Splitting the input into groups
2. Applying `func` to each group
3. Collecting the results into an output group


<img src="https://docs.google.com/drawings/d/e/2PACX-1vSpZlYnXg8MfRHlRjm8JDcxkCjrQfI2XoS06JikaoRCuZiQUUgyo5yjWASU-ynNcucK2-eumooIty1-/pub?w=960&amp;h=720">

Now let's suppose we wanted to speed that up with Numba. As a user, you could `@numba.jit` your function. Depending on what your user defined function is doing, that could lead to a nice speedup. But there would still be a bunch of overhead *around* your function that would be relatively slow. Pandas would need to slice into the array (from Python), call your fast function (now in fast machine code), and jump back to Python to form the output array.

<img src="https://docs.google.com/drawings/d/e/2PACX-1vRwvBtrV51LU2qfOxXUrggJ7h0-bTeSSozatQ7AECyhSOxEdO0ivfoXNhwWM5Q-lZvRBxmPMeAX5hzf/pub?w=960&amp;h=540">

When you use the `engine="numba"` keyword, pandas and Numba able to JIT compile a lot more than just your function. We're able to JIT the entire splitting, function application, and result combination so that the whole things stays in fast machine code.

<img src="https://docs.google.com/drawings/d/e/2PACX-1vRYpI3MI4LKZQSz2VUAxQrxiN6wAlnmTCLOF2VcYTDtF5dJEbSE6IY1MgFH8w8GH84Q2Suu9ngjgYD0/pub?w=960&amp;h=540">

For example, let's compute the mean absolute deviation. Pandas doesn't have a builtin version.

In [51]:
def mad(x):
    return np.fabs(x - x.mean()).mean()

The original dataset `ts` is 10-second frequency. We'll do a rolling mean absolute deviation at 1-minute frequency. But, the naive version is too slow to do on the full dataset.

In [52]:
%%time
# For speed, limit to 10,000 rows
ts[["x", "y"]].head(10_000).rolling("T").apply(mad, raw=True)

CPU times: user 253 ms, sys: 6.56 ms, total: 260 ms
Wall time: 260 ms


,x,y
timestamp,,
2000-01-01 00:00:00,0.000000,0.000000
2000-01-01 00:00:10,0.264913,0.004859
2000-01-01 00:00:20,0.334535,0.145066
2000-01-01 00:00:30,0.286704,0.191164
2000-01-01 00:00:40,0.269602,0.387733
...,...,...
2000-01-02 03:45:50,0.539874,0.477123
2000-01-02 03:46:00,0.525479,0.513446
2000-01-02 03:46:10,0.529432,0.517274


Now let's try this with `engine="numba"`. At first, things don't look great.

In [53]:
%%time
ts[["x", "y"]].head(10_000).rolling("T").apply(
    mad, engine="numba", raw=True
)

CPU times: user 1.07 s, sys: 195 ms, total: 1.27 s
Wall time: 1.59 s


,x,y
timestamp,,
2000-01-01 00:00:00,0.000000,0.000000
2000-01-01 00:00:10,0.264913,0.004859
2000-01-01 00:00:20,0.334535,0.145066
2000-01-01 00:00:30,0.286704,0.191164
2000-01-01 00:00:40,0.269602,0.387733
...,...,...
2000-01-02 03:45:50,0.539874,0.477123
2000-01-02 03:46:00,0.525479,0.513446
2000-01-02 03:46:10,0.529432,0.517274


So the operation is a bit slower. But that's becuase Numba is a just-in-time compiler. It observes what your code is doing and compiles some machine code tailored to the work being done. That compilation takes time, so it's cached and reused.  We can call it again and see that things are even faster on subsequent calls.

In [54]:
%timeit ts[["x", "y"]].head(10_000).rolling("T").apply(mad, engine="numba", raw=True)

100 loops, best of 5: 10.8 ms per loop


Indeed, it's fast enough that we can call it on the whole thing now.

In [56]:
%%timeit
_ = ts[["x", "y"]].rolling("T").apply(
    mad, engine="numba", raw=True
)

1 loop, best of 5: 227 ms per loop


Using numba for user-defined functions passed to pandas' apply, agg, and transform is extremely powerful. In the currently released version of pandas (1.1) numba-accelerated operations are available in

* GroupBy.aggregate
* GroupBy.transform
* Rolling/Expanding.apply
* Rolling/Expanding.aggregate

## Summary

Today we've seen a few strategies for writing high-performance pandas code

1. Choose the best file format for your needs

File formats like Parquet can offer higher performance, especially if your workload only needs to read in subsets of the data

2. Avoid reapeatedly expanding DataFrames along the rows

We saw that repeatedly calling DataFrame.append was slower than building many dataframes and concatentating them at the end.

3. Use the right data type

We saw that using pandas' new nullable types can avoid slow `object`-dtypes and cut down on memory usage.

4. Avoid iteration and apply

We implemented two versions of the great circle distance computation. In pandas, the vectorized version using NumPy was faster than the NumPy version.

5. Use Numba for user-defined functions

Pandas may not always have a built-in version of the method you need.